In [1]:
import pandas as pd
import requests
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
import getpass


In [2]:
username = input("Enter your ig user: ")
password = getpass.getpass("Enter your password: ")

In [4]:
login_url = "https://www.instagram.com/accounts/login/ajax/"

session = requests.Session()

# Get the CSRF token by visiting the login page first
response = session.get("https://www.instagram.com/accounts/login/")
csrf = session.cookies.get("csrftoken")

# Headers required for the login request
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0',
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://www.instagram.com/accounts/login/",
    "x-csrftoken": csrf
}

session.headers.update(headers)

current_time = int(datetime.now().timestamp())

payload = {
    "username": username,
    'enc_password': f'#PWD_INSTAGRAM_BROWSER:0:{current_time}:{password}',
    "queryParams": {},
    "optIntoOneTap": "false"
}

response = session.post(login_url, data=payload)

if response.status_code == 200 and response.json().get("authenticated"):
    print("Login successful.")
else:
    raise("Login failed.")

user_id = response.json()['userId']

Login successful.


In [5]:
session.headers.get("x-csrftoken")

'n_SY5M9WZleDNFlnLTcYud'

In [127]:
users_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0',
    "x-csrftoken": csrf,
    "Accept": "application/json",
    "Content-Type": "application/json",
    'x-asbd-id': '129477',
    'x-ig-app-id': '936619743392459',
    'x-ig-www-claim': 'hmac.AR3QKI1WxD2BGDYqQ2LKCyryMMsIXXQHAT85AC9znp48hi_s',
    'x-requested-with': 'XMLHttpRequest',
}

session.headers.update(users_headers)

In [128]:
response = session.get(
        f'https://www.instagram.com/api/v1/users/{user_id}/info/'
    )

total_following_count = response.json()['user']['following_count']
total_follower_count = response.json()['user']['follower_count']
print(f'Following: {total_following_count}; Followers: {total_follower_count}')

Following: 487; Followers: 595


In [129]:
followers_df = pd.DataFrame()

# initial params
params = {
    'count': '10',
    'search_surface': 'follow_list_page',
}


with tqdm(total=total_follower_count, desc="Fetching Followers", unit=" follower") as pbar:
    for i in range(total_follower_count): # upper bound number of request by of followers
        response = session.get(
            f'https://www.instagram.com/api/v1/friendships/{user_id}/followers/',
            params=params,
        )

        # add followers to df
        current_followers_df = pd.DataFrame(response.json()['users'])[['id', 'username', 'full_name', 'is_private']]
        followers_df = pd.concat([followers_df, current_followers_df], ignore_index=True)
        followers_df.drop_duplicates(inplace=True)

        # Update the progress bar
        pbar.n = len(followers_df)
        pbar.set_postfix({"Number of requests": f"{i}"})
        pbar.refresh()

        # prep for next request or stop break out of loop
        if 'next_max_id' in response.json():
            params['max_id'] = response.json()['next_max_id']
        else:
            break

        # take a chill pill every 10 requests
        if (i + 1) % 10 == 0:
            time.sleep(1)

Fetching Followers: 100%|██████████| 595/595 [00:18<00:00, 31.62 follower/s, Number of requests=65]


In [130]:
following_df = pd.DataFrame()

params = {
    'count': '12',
}

with tqdm(total=total_following_count, desc="Fetching Followings", unit=" followings") as pbar:
    for i in range(total_following_count):
        response = session.get(
            f'https://www.instagram.com/api/v1/friendships/{user_id}/following/',
            params=params,
        )

        # add followers to df
        current_following_df = pd.DataFrame(response.json()['users'])[['id', 'username', 'full_name', 'is_private']]
        following_df = pd.concat([following_df, current_following_df], ignore_index=True)
        following_df.drop_duplicates(inplace=True)

        # Update the progress bar
        pbar.n = len(following_df)
        pbar.set_postfix({"Number of requests": f"{i}"})
        pbar.refresh()

        # prep for next request or stop break out of loop
        if 'next_max_id' in response.json():
            params['max_id'] = response.json()['next_max_id']
        else:
            break

        # take a chill pill every 10 requests
        if (i + 1) % 10 == 0:
            time.sleep(1)


Fetching Followings: 100%|██████████| 487/487 [00:18<00:00, 27.05 followings/s, Number of requests=40]


In [131]:
non_followers_usernames = list(np.setdiff1d(following_df['username'], followers_df['username']))
non_followers_usernames

['davidgoggins',
 'hubermanlab',
 'jordan.b.peterson',
 'lazyprogrammerofficial',
 'reallifeheroes.13',
 'science']

In [132]:
unfollowers_df = following_df[following_df['username'].isin(non_followers_usernames)].copy()
unfollowers_df['link'] = 'https://www.instagram.com/' + following_df['username']
unfollowers_df

,id,username,full_name,is_private,link
480,63481415612,lazyprogrammerofficial,Lazy Programmer - Everything AI/ML & Data Science,False,https://www.instagram.com/lazyprogrammerofficial
481,2357011531,jordan.b.peterson,Dr. Jordan B. Peterson,False,https://www.instagram.com/jordan.b.peterson
482,2093326268,science,Science 🧬,False,https://www.instagram.com/science
483,63218024160,reallifeheroes.13,RealLifeHeroes,False,https://www.instagram.com/reallifeheroes.13
484,4293712500,hubermanlab,"Andrew Huberman, Ph.D.",False,https://www.instagram.com/hubermanlab
485,3544528924,davidgoggins,David Goggins,False,https://www.instagram.com/davidgoggins
